## Generator

In [1]:
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils import data

      
class DCGAN_G(nn.Module):
    """ 
        generator component of WGAN
    """
    def __init__(self, ngf, nz):
        super(DCGAN_G, self).__init__()
        
        self.ngf = ngf
        self.nz = nz

        kernel = 4
        
        # input energy shape [batch x 1 x 1 x 1 ] going into convolutional
        self.conv1_1 = nn.ConvTranspose3d(1, ngf*4, kernel, 1, 0, bias=False)
        # state size [ ngf*4 x 4 x 4 x 4]
        
        # input noise shape [batch x nz x 1 x 1] going into convolutional
        self.conv1_100 = nn.ConvTranspose3d(nz, ngf*4, kernel, 1, 0, bias=False)
        # state size [ ngf*4 x 4 x 4 x 4]
        
        
        # outs from first convolutions concatenate state size [ ngf*8 x 4 x 4]
        # and going into main convolutional part of Generator
        self.main_conv = nn.Sequential(
            
            nn.ConvTranspose3d(ngf*8, ngf*4, kernel, 2, 1, bias=False),
            nn.LayerNorm([8, 8, 8]),
            nn.ReLU(),
            # state shape [ (ndf*4) x 8 x 8 ]

            nn.ConvTranspose3d(ngf*4, ngf*2, kernel, 2, 1, bias=False),
            nn.LayerNorm([16, 16, 16]),
            nn.ReLU(),
            # state shape [ (ndf*2) x 16 x 16 ]

            nn.ConvTranspose3d(ngf*2, ngf, kernel, 2, 1, bias=False),
            nn.LayerNorm([32, 32, 32]),
            nn.ReLU(),
            # state shape [ (ndf) x 32 x 32 ]

            nn.ConvTranspose3d(ngf, 1, 3, 1, 2, bias=False),
            nn.ReLU()
            # state shape [ 1 x 30 x 30 x 30 ]
        )

    def forward(self, noise, energy):
        energy_trans = self.conv1_1(energy)
        noise_trans = self.conv1_100(noise)
        input = torch.cat((energy_trans, noise_trans), 1)
        x = self.main_conv(input)
        x = x.view(-1, 30, 30, 30)
        return x


def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('LayerNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)
    elif classname.find('Linear') != -1:
        m.weight.data.normal_(0.0, 0.02)
        m.bias.data.fill_(0)

In [2]:
import numpy as np

In [34]:
device = torch.device("cuda")

BATCH_SIZE = 10
LATENT = 50

noise = np.random.uniform(-1, 1, (BATCH_SIZE, LATENT))    
noise = torch.from_numpy(noise).float()
noise = noise.view(-1, LATENT, 1, 1, 1)    #[BS, nz]  --> [Bs,nz,1,1,1] Needed for Generator
noise = noise.to(device)    

In [35]:
real_labelnp = np.random.uniform(10, 100, (BATCH_SIZE,1,1,1,1))
real_label = torch.from_numpy(real_labelnp).float()
real_label = real_label.to(device)

In [40]:
aG = DCGAN_G(16, LATENT).to(device)

with torch.no_grad():
    noisev = noise  # totally freeze G, training D

fake_data = aG(noisev, real_label).detach()
            

In [41]:
fake_data.shape

torch.Size([10, 30, 30, 30])

In [43]:
sum(p.numel() for p in aG.parameters() if p.requires_grad)
#aG

972208

## Data Loader 

In [7]:
import h5py
import torch
from torch.utils import data
import numpy as np



class HDF5Dataset(data.Dataset):
    def __init__(self, file_path, train_size, transform=None):
        super().__init__()
        self.file_path = file_path
        self.transform = transform
        self.hdf5file = h5py.File(self.file_path, 'r')
        
        if train_size > self.hdf5file['ecal']['layers'].shape[0]-1:
            self.train_size = self.hdf5file['ecal']['layers'].shape[0]-1
        else:
            self.train_size = train_size
            
        
    def __len__(self):
        return self.hdf5file['ecal']['layers'][0:self.train_size].shape[0]
             
    def __getitem__(self, index):
        # get data
        x = self.get_data(index)
        if self.transform:
            x = torch.from_numpy(self.transform(x)).float()
        else:
            x = torch.from_numpy(x).float()
        e = torch.from_numpy(self.get_energy(index))
        if torch.sum(x) != torch.sum(x): #checks for NANs
            return self.__getitem__(int(np.random.rand()*self.__len__()))
        else:
            return x, e
    
    def get_data(self, i):
        return self.hdf5file['ecal']['layers'][i]
    
    def get_energy(self, i):
        return self.hdf5file['ecal']['energy'][i]

In [57]:
device = torch.device("cuda")
dataset = HDF5Dataset('/eos/user/e/eneren/run_prod4k/mergedData_prod3k.hdf5', transform=None, train_size=3000)

loader_params = {'shuffle': True, 'num_workers': 2}

train_loader = torch.utils.data.DataLoader(dataset, batch_size = 100, drop_last=True, **loader_params)


In [58]:
for batch_idx, (data, energy) in enumerate(train_loader):
    print (data.shape, energy.shape)

torch.Size([100, 30, 30, 30]) torch.Size([100, 1])
torch.Size([100, 30, 30, 30]) torch.Size([100, 1])
torch.Size([100, 30, 30, 30]) torch.Size([100, 1])
torch.Size([100, 30, 30, 30]) torch.Size([100, 1])
torch.Size([100, 30, 30, 30]) torch.Size([100, 1])
torch.Size([100, 30, 30, 30]) torch.Size([100, 1])
torch.Size([100, 30, 30, 30]) torch.Size([100, 1])
torch.Size([100, 30, 30, 30]) torch.Size([100, 1])
torch.Size([100, 30, 30, 30]) torch.Size([100, 1])
torch.Size([100, 30, 30, 30]) torch.Size([100, 1])
torch.Size([100, 30, 30, 30]) torch.Size([100, 1])
torch.Size([100, 30, 30, 30]) torch.Size([100, 1])
torch.Size([100, 30, 30, 30]) torch.Size([100, 1])
torch.Size([100, 30, 30, 30]) torch.Size([100, 1])
torch.Size([100, 30, 30, 30]) torch.Size([100, 1])
torch.Size([100, 30, 30, 30]) torch.Size([100, 1])
torch.Size([100, 30, 30, 30]) torch.Size([100, 1])
torch.Size([100, 30, 30, 30]) torch.Size([100, 1])
torch.Size([100, 30, 30, 30]) torch.Size([100, 1])
torch.Size([100, 30, 30, 30]) t

## Critic 

In [16]:
from criticRes import *
#import criticRes

In [18]:
aD = generate_model(18).to(device)
for batch_idx, (data, energy) in enumerate(train_loader):
    #print (data.shape, energy.shape)
    data = data.to(device).unsqueeze(1)  
    real_label = energy.to(device)
    calo_size = data.size(-1)
    
    disc_real = aD(data.float(), real_label.float()) 
    print (disc_real.shape)

torch.Size([100, 1])
torch.Size([100, 1])
torch.Size([100, 1])
torch.Size([100, 1])
torch.Size([100, 1])
torch.Size([100, 1])
torch.Size([100, 1])
torch.Size([100, 1])
torch.Size([100, 1])
torch.Size([100, 1])


In [21]:
sum(p.numel() for p in aD.parameters() if p.requires_grad)


2005378